In [ ]:
from herbie.archive import Herbie
from herbie.tools import fast_Herbie, fast_Herbie_download, fast_Herbie_xarray
import pandas as pd

import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cmaps

import geocat.viz as gv
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.animation as animation


# convert the animation to javascript for display
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['animation.embed_limit'] = 2**128
import geocat.viz.util as gvutil

In [ ]:
DATES = pd.date_range('2022-08-30', periods=6, freq='6H')
DATES

In [ ]:
# Create a list of forecast lead times
fxx = [0]
#HH = fast_Herbie(DATES=DATES, fxx=fxx, model='gfs',product='pgrb2.0p25',priority='aws')




#H = Herbie('2022-9-02 00:00', model='gfs',
#           product='pgrb2.0p25',priority='aws', fxx=0)



#Hu = fast_Herbie_xarray('2022-9-02 00:00', model='gfs',
#           product='pgrb2.0p25',priority='aws', fxx=fxx, searchString='UGRD:850 mb')



In [ ]:
Hu=  fast_Herbie_xarray(DATES=DATES, fxx=fxx, model='gfs',product='pgrb2.0p25',priority='aws',searchString='UGRD:850 mb')

In [ ]:
Hv =  fast_Herbie_xarray(DATES=DATES, fxx=fxx, model='gfs',product='pgrb2.0p25',priority='aws',searchString='VGRD:850 mb')

In [ ]:
import metpy.calc 

# Any import of metpy will activate the accessors
from metpy.units import units
#
from metpy.calc import vorticity



min_lon = 260.
max_lon =  360.
delta_lon = 20.
min_lat =  0.
max_lat =  30.

u = Hu.u.sel(latitude=slice(min_lat, max_lat), longitude=slice(min_lon, max_lon))
v = Hv.v.sel(latitude=slice(min_lat, max_lat), longitude=slice(min_lon, max_lon)) 

vor= vorticity(Hu.u,Hv.v).sel(latitude=slice(max_lat, min_lat), longitude=slice(min_lon, max_lon)) 
vor=vor*1.e5


In [ ]:
#vor


In [ ]:

# Generate figure and set its size (width, height) in inches
fig = plt.figure(figsize=(10,8))

# Generate axes, using Cartopy, drawing coastlines, and adding features
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(linewidths=0.5)
ax.add_feature(cfeature.LAND, facecolor='lightgray')

cf = vor[0,:,:].plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-20, 20, 2),
                          cmap='bwr',extend='both',
                          add_colorbar=False)
# Add colorbar
cbar = plt.colorbar(cf,
                    shrink=0.8,
                    drawedges=True,
                    ticks=np.arange(-20,20,4),
                    extendrect=True,
                    extendfrac='auto', orientation='horizontal')

lon1tick=280
lon2tick=340

gl = ax.gridlines(draw_labels=True)
#ax.coastlines('50m', linewidth=0.8,color='gray')
gl.top_labels = gl_right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER


# animate function for matplotlib FuncAnimation
def animate(i):
    print ( "now ", i)
    projection = ccrs.PlateCarree()
    cf = vor[i,:,:].plot.contourf(ax=ax,
                          transform=projection,
                          levels=np.arange(-20, 20, 2),
                          cmap='bwr',extend='both',
                          add_colorbar=False)

# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig, animate, frames=6,  interval=200)
# Uncomment this line to save the created animation
#anim.save('animate_1.gif', writer='pillow', fps=5)

anim